## Aspect Reviews Recommander

This file provide a function aiming to providing useful reviews for user by inputting restaurantId and a particular aspect name.

#### 1. Prepare dataset

1) load review dataset, get columns 'review_id', 'categories', 'text','RestaurantsPriceRange2', 'RestaurantsAttire' from this dataset.
2) Load aspects dataset which is the output result of deberta model, precess it and aggragate with the review dataset.

In [1]:
import pandas as pd
import numpy as np
 
aspect_result_path = '../data/deberta_aspect_score_sentiment_results.csv'
ca_file_path = '../data/ca_restaurants.csv'
ca_df = pd.read_csv(ca_file_path)
aspects_df = pd.read_csv(aspect_result_path)
aspects_df

,Unnamed: 0,business_id,review_id,ambiance_label,atmosphere_label,dessert_label,drinks_label,entertainment_label,experience_label,food_label,...,kid_friendly_score,location_score,noise_level_score,price_score,seating_score,service_score,setting_score,space_score,utilities_score,waiting_time_score
0,0,-3AooxIkg38UyUdlz5oXdw,Bl1VqhkkGx7EoUrDOCFW8w,NaN,Negative,NaN,NaN,NaN,NaN,Positive,...,NaN,0.517108,NaN,NaN,NaN,0.453472,NaN,0.537013,NaN,0.680859
1,1,-3AooxIkg38UyUdlz5oXdw,KiU6B5XoFdT9l3MsGYPOZg,NaN,NaN,NaN,NaN,NaN,Positive,Positive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.892360
2,2,-3AooxIkg38UyUdlz5oXdw,V7Wt6sPCTGAqbYs8okNQ3w,NaN,Positive,NaN,NaN,NaN,NaN,Negative,...,NaN,0.600989,NaN,0.831413,NaN,0.501830,0.630508,0.539700,0.574026,NaN
3,3,-8iATYRnN46Km0_-ldx6cg,8xVsO5itmEY0xZ3u898AMg,NaN,Positive,NaN,Positive,NaN,NaN,Positive,...,NaN,0.999852,NaN,NaN,NaN,NaN,0.914718,0.992303,NaN,NaN
4,4,-9r8nAzWyRSLxBWt8uQOdA,8kheTuGA9sPd6MVyXRr6Kw,Positive,Positive,NaN,NaN,NaN,NaN,Positive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,1848,znlrxrY4InSx7ekPZxoHRw,zrif4eelRzkxnTcc5pNKiw,NaN,Positive,NaN,NaN,NaN,NaN,Positive,...,NaN,0.995302,NaN,NaN,NaN,NaN,0.952239,0.987540,NaN,NaN
1849,1849,zpuFEeAhrNzXPkOlBf5Kog,lrvtEz4KpJhV8LUXL-mbuA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.996491,NaN,0.999866,NaN,NaN,0.931868,NaN
1850,1850,zpuFEeAhrNzXPkOlBf5Kog,v2VPYuS7Zkoqg6AkHjzc3Q,NaN,Negative,NaN,NaN,NaN,NaN,NaN,...,NaN,0.550732,NaN,NaN,NaN,NaN,NaN,0.677887,NaN,NaN
1851,1851,zu4p6IZLSVn2Noto-vcwzw,3Kinrkj2zeWKxh3nXyOfdw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.750418,NaN,NaN,NaN,NaN,NaN,0.666970,NaN,0.581326


In [2]:
import ast
from pandas import json_normalize
def prepare_review_df(df):
    """
    Prepare the reviews of CA
    :param df: reviews dataframe 
    :return: ca_df with flatten "attributes" colums
    """
    review_df = df.copy()
    review_df['attributes'] = review_df['attributes'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
    #  flatten Json 
    flattened_attributes = json_normalize(review_df['attributes'])
    # Combine the flattened attributes with the original DataFrame
    review_df = pd.concat([review_df, flattened_attributes], axis=1)
    return review_df
review_df = prepare_review_df(ca_df)
review_df 

,Unnamed: 0,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,...,Music,CoatCheck,BestNights,GoodForDancing,Smoking,AgesAllowed,RestaurantsCounterService,AcceptsInsurance,Open24Hours,DietaryRestrictions
0,1375,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1376,_Ub20uO1MKy4XOVPOdzpqw,lUYboGI6aFbZ0dX27pijpA,gebiRewfieSdtt17PTW6Zg,1,0,2,0,"Sat down over a half hour, only miso and drink...",2017-06-28 01:04:59.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1377,-DjIfoNFAiT5J4kF9hXocQ,SrfDRvGKI8FQq9LCr0dQuQ,gebiRewfieSdtt17PTW6Zg,3,9,4,6,Not bad. Sushi roll was good but options were ...,2017-01-14 23:31:35.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1378,URe-2Zlv7RoY4Ycr9JYVWw,SfMf7DEX7xT82OCcbZFsgQ,gebiRewfieSdtt17PTW6Zg,5,1,0,1,I'm very picky when it comes to Sushi. Quality...,2016-07-14 23:09:38.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1379,tHP5ihPd_W9OzfkKEWdIwA,aH3AH7V9g60w4pt3LpJ1_Q,gebiRewfieSdtt17PTW6Zg,4,0,0,0,Even though the sushi masters are not actually...,2014-11-30 07:35:54.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247608,6990221,cKuP5pnvbvpn4UbaccVcdg,OVYb1El25BgVhgbXb4e-KA,XPkkWyAkkPeH-PqXmBMmpg,5,0,0,0,Great experience with great people who are rea...,2021-06-22 21:41:07.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247609,6990222,dZC87sZ1f2ntOf876_DDLg,9w5PoliouEyNikdy6DizqQ,XPkkWyAkkPeH-PqXmBMmpg,5,0,0,0,I spent the week in Santa Barbara and had the ...,2021-06-28 23:53:44.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247610,6990223,NCafkcXIvK7E965my4RpQg,GmuHOkMR80oW3sAuF-UsAw,XPkkWyAkkPeH-PqXmBMmpg,5,0,0,0,"Last summer, I had the pleasure of experiencin...",2021-07-06 19:15:56.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247611,6990224,VxsNftGxRUDZcLD0Ns1wKg,VRXPOvlSydho0tYmSseKnQ,XPkkWyAkkPeH-PqXmBMmpg,5,0,0,0,Joseph and Teresa are super hosts. You will f...,2021-07-01 03:44:10.000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# get final df
df = pd.merge(aspects_df, review_df[['review_id', 'categories', 'text','RestaurantsPriceRange2', 'RestaurantsAttire']], on='review_id', how='left')
df

,Unnamed: 0,business_id,review_id,ambiance_label,atmosphere_label,dessert_label,drinks_label,entertainment_label,experience_label,food_label,...,seating_score,service_score,setting_score,space_score,utilities_score,waiting_time_score,categories,text,RestaurantsPriceRange2,RestaurantsAttire
0,0,-3AooxIkg38UyUdlz5oXdw,Bl1VqhkkGx7EoUrDOCFW8w,NaN,Negative,NaN,NaN,NaN,NaN,Positive,...,NaN,0.453472,NaN,0.537013,NaN,0.680859,"Event Planning & Services, Restaurants, Nightl...",Had friends come in from out of town and they ...,2,'casual'
1,1,-3AooxIkg38UyUdlz5oXdw,KiU6B5XoFdT9l3MsGYPOZg,NaN,NaN,NaN,NaN,NaN,Positive,Positive,...,NaN,NaN,NaN,NaN,NaN,0.892360,"Event Planning & Services, Restaurants, Nightl...",My friend and I went to Santa Barbara for our ...,2,'casual'
2,2,-3AooxIkg38UyUdlz5oXdw,V7Wt6sPCTGAqbYs8okNQ3w,NaN,Positive,NaN,NaN,NaN,NaN,Negative,...,NaN,0.501830,0.630508,0.539700,0.574026,NaN,"Event Planning & Services, Restaurants, Nightl...","2.5-3 Overall\n\nPrice\nIt's Santa Barbara, so...",2,'casual'
3,3,-8iATYRnN46Km0_-ldx6cg,8xVsO5itmEY0xZ3u898AMg,NaN,Positive,NaN,Positive,NaN,NaN,Positive,...,NaN,NaN,0.914718,0.992303,NaN,NaN,"Food, Restaurants, American (New)",What's not to love about this place? I LOVE T...,2,u'casual'
4,4,-9r8nAzWyRSLxBWt8uQOdA,8kheTuGA9sPd6MVyXRr6Kw,Positive,Positive,NaN,NaN,NaN,NaN,Positive,...,NaN,NaN,NaN,NaN,NaN,NaN,"Vegan, Asian Fusion, Bubble Tea, Restaurants, ...",The chicken bowl is definitely a must try! I a...,1,u'casual'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,1848,znlrxrY4InSx7ekPZxoHRw,zrif4eelRzkxnTcc5pNKiw,NaN,Positive,NaN,NaN,NaN,NaN,Positive,...,NaN,NaN,0.952239,0.987540,NaN,NaN,"Food, Ice Cream & Frozen Yogurt, Desserts",This is my favorite place for ice cream and I ...,2,'casual'
1849,1849,zpuFEeAhrNzXPkOlBf5Kog,lrvtEz4KpJhV8LUXL-mbuA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.999866,NaN,NaN,0.931868,NaN,"Steakhouses, Restaurants",There is a sign on the door letting you know t...,2,'casual'
1850,1850,zpuFEeAhrNzXPkOlBf5Kog,v2VPYuS7Zkoqg6AkHjzc3Q,NaN,Negative,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.677887,NaN,NaN,"Steakhouses, Restaurants",why o why o why would you ever go to Outback w...,2,'casual'
1851,1851,zu4p6IZLSVn2Noto-vcwzw,3Kinrkj2zeWKxh3nXyOfdw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.666970,NaN,0.581326,"Breakfast & Brunch, Restaurants, American (Tra...","went on a beautiful day, right on the beach. D...",2,u'casual'


#### 2. Filter similar restaurants by Price Range and Restaurant Category Descriptions
2 key steps:
1) filter by price range (column RestaurantsPriceRange2). if it does not have a specific range number(Nan), return all restaurants
2) get cosine similarity higher than 0.65 based on restaurant type categories.

In [4]:
# get unique restaurants in the dataset
res_df = df[['business_id', 'categories', 'RestaurantsPriceRange2', 'RestaurantsAttire']].drop_duplicates()
res_df

,business_id,categories,RestaurantsPriceRange2,RestaurantsAttire
0,-3AooxIkg38UyUdlz5oXdw,"Event Planning & Services, Restaurants, Nightl...",2,'casual'
3,-8iATYRnN46Km0_-ldx6cg,"Food, Restaurants, American (New)",2,u'casual'
4,-9r8nAzWyRSLxBWt8uQOdA,"Vegan, Asian Fusion, Bubble Tea, Restaurants, ...",1,u'casual'
7,-BdYhP-12elmFV7oB1iv4A,"Restaurants, Gastropubs, Cocktail Bars, Bars, ...",NaN,NaN
8,-FM4CxOg4XXmX_Ebky_SiQ,"Breakfast & Brunch, American (New), Restaurant...",2,u'casual'
...,...,...,...,...
1834,ziXR7sUrbKRCNeDqfjibUg,"Restaurants, Mexican, Breakfast & Brunch",1,'casual'
1836,zkMqKD6IZPcU10g0a2APmg,"Wine Bars, Bars, Shopping, Beer Bar, Tobacco S...",2,NaN
1837,znlrxrY4InSx7ekPZxoHRw,"Food, Ice Cream & Frozen Yogurt, Desserts",2,'casual'
1849,zpuFEeAhrNzXPkOlBf5Kog,"Steakhouses, Restaurants",2,'casual'


In [5]:
# functions for get name and id by each other
restaurant_names_and_ids = ca_df[['name', 'business_id']].drop_duplicates()
restaurant_names_ids = restaurant_names_and_ids.copy()
restaurant_names_ids.set_index('name', inplace=True)
restaurant_ids_names = restaurant_names_and_ids.copy() 
restaurant_ids_names.set_index('business_id', inplace=True)
def get_resaurant_id_by_name(name):
    name = name.strip()
    return restaurant_names_ids.loc[name]['business_id']
def get_resaurant_name_by_id(id):
    id = id.strip()
    return restaurant_ids_names.loc[id]['name']

In [6]:
# input_id = get_resaurant_id_by_name("Corner Tap Room")
input_asp = "kid friendly"
input_id = "-H-fZonTLitwHFY_PLAIjA"
def filter_restaurants_by_price(input_id):
    #1.filter restaurants at the same price segment, we only keep restaurants in the same price range
    input_row = res_df[res_df['business_id'] == input_id]
    price_segment = input_row['RestaurantsPriceRange2'].iloc[0]
    if not pd.isna(price_segment):
        filtered_res = res_df[res_df['RestaurantsPriceRange2'] == price_segment].copy()
    else:
        filtered_res = res_df.copy()
    filtered_res['RestaurantsAttire'] = filtered_res['RestaurantsAttire'].astype(str)
    return filtered_res
filtered_res = filter_restaurants_by_price(input_id)
filtered_res

,business_id,categories,RestaurantsPriceRange2,RestaurantsAttire
0,-3AooxIkg38UyUdlz5oXdw,"Event Planning & Services, Restaurants, Nightl...",2,'casual'
3,-8iATYRnN46Km0_-ldx6cg,"Food, Restaurants, American (New)",2,u'casual'
8,-FM4CxOg4XXmX_Ebky_SiQ,"Breakfast & Brunch, American (New), Restaurant...",2,u'casual'
22,-H-fZonTLitwHFY_PLAIjA,"Restaurants, Pizza",2,u'casual'
24,-I7M6l0h46VdiN09Y1NUZw,"Gluten-Free, Gastropubs, Vegan, Restaurants, I...",2,nan
...,...,...,...,...
1831,zeAfmYy9b1gfUJLRBHm6vQ,"Restaurants, Italian",2,u'casual'
1836,zkMqKD6IZPcU10g0a2APmg,"Wine Bars, Bars, Shopping, Beer Bar, Tobacco S...",2,nan
1837,znlrxrY4InSx7ekPZxoHRw,"Food, Ice Cream & Frozen Yogurt, Desserts",2,'casual'
1849,zpuFEeAhrNzXPkOlBf5Kog,"Steakhouses, Restaurants",2,'casual'


In [7]:
df['RestaurantsAttire'].unique()

array(["'casual'", "u'casual'", nan, "u'dressy'", "'dressy'"],
      dtype=object)

In [8]:
# clean RestaurantsAttire column, only keep casual, dressy key words, remove ' ', NaN, and u' ' 
def create_corpus(x):
    format_word = lambda word: "" if word == "nan" else " " + word
    res_attr = x['RestaurantsAttire'].strip("u").strip("'")
    categories = [cate for cate in x['categories'].split(', ') if cate not in ['Restaurants','Food']]
    return ' '.join(categories) + format_word(res_attr)
filtered_res['corpus'] = filtered_res.apply(create_corpus, axis=1)
filtered_res.reset_index(inplace=True)
filtered_res

,index,business_id,categories,RestaurantsPriceRange2,RestaurantsAttire,corpus
0,0,-3AooxIkg38UyUdlz5oXdw,"Event Planning & Services, Restaurants, Nightl...",2,'casual',Event Planning & Services Nightlife Caterers I...
1,3,-8iATYRnN46Km0_-ldx6cg,"Food, Restaurants, American (New)",2,u'casual',American (New) casual
2,8,-FM4CxOg4XXmX_Ebky_SiQ,"Breakfast & Brunch, American (New), Restaurant...",2,u'casual',Breakfast & Brunch American (New) American (Tr...
3,22,-H-fZonTLitwHFY_PLAIjA,"Restaurants, Pizza",2,u'casual',Pizza casual
4,24,-I7M6l0h46VdiN09Y1NUZw,"Gluten-Free, Gastropubs, Vegan, Restaurants, I...",2,nan,Gluten-Free Gastropubs Vegan Indian
...,...,...,...,...,...,...
458,1831,zeAfmYy9b1gfUJLRBHm6vQ,"Restaurants, Italian",2,u'casual',Italian casual
459,1836,zkMqKD6IZPcU10g0a2APmg,"Wine Bars, Bars, Shopping, Beer Bar, Tobacco S...",2,nan,Wine Bars Bars Shopping Beer Bar Tobacco Shops...
460,1837,znlrxrY4InSx7ekPZxoHRw,"Food, Ice Cream & Frozen Yogurt, Desserts",2,'casual',Ice Cream & Frozen Yogurt Desserts casual
461,1849,zpuFEeAhrNzXPkOlBf5Kog,"Steakhouses, Restaurants",2,'casual',Steakhouses casual


In [9]:
#create indices to locate indices and rest_id
indices = pd.Series(filtered_res.index, index=filtered_res['business_id']).drop_duplicates()
indices

business_id
-3AooxIkg38UyUdlz5oXdw      0
-8iATYRnN46Km0_-ldx6cg      1
-FM4CxOg4XXmX_Ebky_SiQ      2
-H-fZonTLitwHFY_PLAIjA      3
-I7M6l0h46VdiN09Y1NUZw      4
                         ... 
zeAfmYy9b1gfUJLRBHm6vQ    458
zkMqKD6IZPcU10g0a2APmg    459
znlrxrY4InSx7ekPZxoHRw    460
zpuFEeAhrNzXPkOlBf5Kog    461
zu4p6IZLSVn2Noto-vcwzw    462
Length: 463, dtype: int64

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 2.Compute the Cosine Similarity matrix based on the count_matrix
def computer_cosine_sim_matrix(corpus):
    
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(corpus)
    
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    return cosine_sim
cosine_sim = computer_cosine_sim_matrix(filtered_res['corpus'])
cosine_sim

array([[1.        , 0.20412415, 0.31980107, ..., 0.14433757, 0.25      ,
        0.15811388],
       [0.20412415, 1.        , 0.69631062, ..., 0.23570226, 0.40824829,
        0.51639778],
       [0.31980107, 0.69631062, 1.        , ..., 0.12309149, 0.21320072,
        0.80903983],
       ...,
       [0.14433757, 0.23570226, 0.12309149, ..., 1.        , 0.28867513,
        0.18257419],
       [0.25      , 0.40824829, 0.21320072, ..., 0.28867513, 1.        ,
        0.31622777],
       [0.15811388, 0.51639778, 0.80903983, ..., 0.18257419, 0.31622777,
        1.        ]])

In [11]:
# 3. get most similar restaurants for the target restaurant
def similar_restaurants(indices, cosine_sim, input_id, df):
    
    idx = indices[input_id]
    # Get the pairwsie similarity scores of all filtered restaurant with the input restaurant
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the restaurants with similarity score above 0.7
    sim_scores = [pair for pair in sim_scores[1:] if pair[1] >= 0.7]
    
    # Get the restaurants indices
    res_indices = [i[0] for i in sim_scores]
    print("There are {} similar unique restaurants found".format(len(res_indices)))
    # get the most similar restaurants id in a list
    most_similar_res = df['business_id'].iloc[res_indices].tolist()
    return most_similar_res
most_similar_res = similar_restaurants(indices, cosine_sim, input_id, filtered_res)

There are 14 similar unique restaurants found


#### 3. Get positive review for the silimar restaurants
Sort by input aspect confidence score, Get positive reviews from the similar restaurants

In [12]:
# 4. output reviews and restaurants
def most_similar_recommendations(most_similar_res, input_asp):
    # in the aspect dataframe aspect_df, only get reviews for most similar restaurant. For these review, filter input aspect with "Positive" and high confidence score
    input_asp = "_".join(input_asp.lower().strip().split(" "))
    temp = df[(df['business_id'].isin(most_similar_res)) & (df[input_asp + '_label'] == 'Positive')].sort_values(by=input_asp + '_score', ascending=False)
    # get the top 10 reviews with their restaurant_id and text review
    top_10_tuples = [(row['business_id'], row['text']) for index, row in temp.head(10).iterrows()]
    for tuple in top_10_tuples:
        print("Review: {}".format(tuple[1]))
        print("Resaurant id: {}".format(tuple[0]))
        print("Restaurant name: {}".format(get_resaurant_name_by_id(tuple[0])))
        print("==================================")
    return top_10_tuples
most_similar_recommendations(most_similar_res, input_asp)

Review: Delicious authentic Italian food, friendly and professional waitstaff. Portions are not huge (but do we really need that much pasta anyway? ). Nice yet comfortable atmosphere and kid friendly. We have been going to Via Vai since we moved to the area almost two years ago and will continue to do so.
Resaurant id: e0DU7_0mfalr-lnyUafiag
Restaurant name: Via Vai Trattoria Pizzeria


[('e0DU7_0mfalr-lnyUafiag',
  'Delicious authentic Italian food, friendly and professional waitstaff. Portions are not huge (but do we really need that much pasta anyway? ). Nice yet comfortable atmosphere and kid friendly. We have been going to Via Vai since we moved to the area almost two years ago and will continue to do so.')]

#### 4. Recommender Function for reviews 
input the restaurant id and aspect name, will get positive feedbacks from most similar restaurants

In [13]:
def get_reviews_recommender(res_id, asp):
    """"
    This function accept a restaurant id and aspect owners care about, 
    return 10 reviews with positive and high confidence score from similar restaurants.
    
    """
    #1. Filter rest by price range
    filtered_res = filter_restaurants_by_price(res_id)
    
    filtered_res['corpus'] = filtered_res.apply(create_corpus, axis=1)
    filtered_res.reset_index(inplace=True)
    
    indices = pd.Series(filtered_res.index, index=filtered_res['business_id']).drop_duplicates()
    
    
    # 2.Compute the Cosine Similarity matrix based on the count_matrix
    cosine_sim = computer_cosine_sim_matrix(filtered_res['corpus'])
    
    # 3. get most similar restaurants for the target restaurant
    most_similar_res = similar_restaurants(indices, cosine_sim, res_id, filtered_res)
    print(most_similar_recommendations(most_similar_res, asp))
    
   # 4. Output result
    
    # return most_similar_recommendations(most_similar_res, asp)

#### Examples:

In [14]:
get_reviews_recommender('zu4p6IZLSVn2Noto-vcwzw', 'atmosphere')

There are 26 similar unique restaurants found
Review: I love Joe's on a weekend evening! It's a great place to start the night. They make stiff drinks and the social scene is popping. Frankie is the BEST bartender! You're in great hands and great company if you hang with him at the bar. This is a great local spot if you're looking to meet new people and enjoy a lively atmosphere.
Resaurant id: nB0dV8yPv29F3-CFMPG8cw
Restaurant name: Joe's Cafe
Review: One of Santa Barbara's best places. 
 Playful, delicious food. Very nice service. Good cocktails and wine/beer selection. 
For breakfast,  their  lemon ricotta pancakes are lovely ( but not available for brunch). 
 
Comfortable and elegant, the atmosphere is perfect for a date, a solo lunch,  a meeting, or a big group.
Resaurant id: -FM4CxOg4XXmX_Ebky_SiQ
Restaurant name: Finch & Fork
Review: Amazing food and the sweetest waitress! Only bad comment was the bloody Mary's weren't good but the food and atmosphere made up for it!
Resaurant id

In [15]:
get_reviews_recommender('zu4p6IZLSVn2Noto-vcwzw', 'waiting_time')

There are 26 similar unique restaurants found
Review: Amazing food--seasoned hasbrowns, loaded baked potato hash, blueberry pancakes...

Can be busy as it is a small place, difficult for groups bigger than 4. Aggressive self seating. Worth the wait.
Resaurant id: 5vObJIHLO-QOMrw_MvC4rA
Restaurant name: The Worker Bee Cafe
Review: Short Rib and Sweet Potato Hash Breakfast!!

We were returning back from the bay area and the little one found this place on Yelp. The restaurant itself sits inside a hotel. It was early and no wait whatsoever. We parked out front on the street and were eating within fifteen minutes.

~Short Rib and Sweet Potato Hash- 4 Stars!
The meat was super tender and juicy. The sweet potato hash, while very tasty, was dry. It was a little too dry. The dish will run you close to $20. Would I order this again? Maybe. The menu has several items I wanted to try.

~Kids Menu
The little one had French toast..of course! She liked it but it looked average to me.

~Servicer- 5 St